<a href="https://colab.research.google.com/github/fantastic-jpn/monday-4th-report/blob/main/report_4th_5th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 2.7 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/PacktPublishing/Hands-On-Genetic-Algorithms-with-Python.git

Cloning into 'Hands-On-Genetic-Algorithms-with-Python'...
remote: Enumerating objects: 246, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 246 (delta 23), reused 17 (delta 17), pack-reused 213 (from 1)
Receiving objects: 100% (246/246), 175.99 KiB | 2.32 MiB/s, done.
Resolving deltas: 100% (136/136), done.


In [ ]:
!git clone https://github.com/ryanjoneil/tsplib.git

Cloning into 'tsplib'...
remote: Enumerating objects: 269, done.
remote: Counting objects: 100% (269/269), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 269 (delta 34), reused 269 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (269/269), 3.01 MiB | 8.06 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [ ]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 27.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
xarray 2025.3.1 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
bigframes 2.4.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
blosc2 3.3.2 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is inco

In [ ]:
from deap import tools
from deap import base
from deap import creator
from deap import algorithms

import math
import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import gym
import time

import pickle

from sklearn.neural_network import MLPRegressor

from sklearn.exceptions import ConvergenceWarning
from sklearn.utils._testing import ignore_warnings

In [ ]:
class CartPole:

    def __init__(self, randomSeed=None):

        self.env = gym.make('CartPole-v1', new_step_api=True)

        if randomSeed is not None:
            self.env.reset(seed=randomSeed)

    def __len__(self):
        return INPUTS * HIDDEN_LAYER + HIDDEN_LAYER * OUTPUTS + HIDDEN_LAYER + OUTPUTS

    @ignore_warnings(category=ConvergenceWarning)
    def initMlp(self, netParams):
        """
        initializes a MultiLayer Perceptron (MLP) Regressor with the desired network architecture (layers)
        and network parameters (weights and biases).
        :param netParams: a list of floats representing the network parameters (weights and biases) of the MLP
        :return: initialized MLP Regressor
        """

        # create the initial MLP:
        mlp = MLPRegressor(hidden_layer_sizes=(HIDDEN_LAYER,), max_iter=1)

        # This will initialize input and output layers, and nodes weights and biases:
        # we are not otherwise interested in training the MLP here, hence the settings max_iter=1 above
        mlp.fit(np.random.uniform(low=-1, high=1, size=INPUTS).reshape(1, -1), np.ones(OUTPUTS))

        # weights are represented as a list of 2 ndarrays:
        # - hidden layer weights: INPUTS x HIDDEN_LAYER
        # - output layer weights: HIDDEN_LAYER x OUTPUTS
        numWeights = INPUTS * HIDDEN_LAYER + HIDDEN_LAYER * OUTPUTS
        weights = np.array(netParams[:numWeights])
        mlp.coefs_ = [
            weights[0:INPUTS * HIDDEN_LAYER].reshape((INPUTS, HIDDEN_LAYER)),
            weights[INPUTS * HIDDEN_LAYER:].reshape((HIDDEN_LAYER, OUTPUTS))
        ]

        # biases are represented as a list of 2 ndarrays:
        # - hidden layer biases: HIDDEN_LAYER x 1
        # - output layer biases: OUTPUTS x 1
        biases = np.array(netParams[numWeights:])
        mlp.intercepts_ = [biases[:HIDDEN_LAYER], biases[HIDDEN_LAYER:]]

        return mlp

    def getScore(self, netParams):
        """
        calculates the score of a given solution, represented by the list of float-valued network parameters,
        by creating a corresponding MLP Regressor, initiating an episode of the Cart-Pole environment and
        running it with the MLP controlling the actions, while using the observations as inputs.
        Higher score is better.
        :param netParams: a list of floats representing the network parameters (weights and biases) of the MLP
        :return: the calculated score value
        """

        mlp = self.initMlp(netParams)

        self.env.reset()

        actionCounter = 0
        totalReward = 0
        observation = self.env.reset()
        action = int(mlp.predict(observation.reshape(1, -1)) > 0)

        while True:
            actionCounter += 1
            observation, reward, terminated, truncated, info = self.env.step(action)
            totalReward += reward

            if terminated or truncated:
                break
            else:
                action = int(mlp.predict(observation.reshape(1, -1)) > 0)
                #print(action)

        return totalReward

    def saveParams(self, netParams):
        """
        serializes and saves a list of network parameters using pickle
        :param netParams: a list of floats representing the network parameters (weights and biases) of the MLP
        """
        savedParams = []
        for param in netParams:
            savedParams.append(param)

        pickle.dump(savedParams, open("cart-pole-data.pickle", "wb"))

    def replayWithSavedParams(self):
        """
        deserializes a saved list of network parameters and uses it to replay an episode
        """
        savedParams = pickle.load(open("cart-pole-data.pickle", "rb"))
        self.replay(savedParams)

    def replay(self, netParams):
        """
        renders the environment and uses the given network parameters to replay an episode, to visualize a given solution
        :param netParams: a list of floats representing the network parameters (weights and biases) of the MLP
        """
        mlp = self.initMlp(netParams)

        self.env.render()

        actionCounter = 0
        totalReward = 0
        observation = self.env.reset()
        action = int(mlp.predict(observation.reshape(1, -1)) > 0)

        while True:
            actionCounter += 1
            self.env.render()
            observation, reward, terminated, truncated, info = self.env.step(action)
            totalReward += reward

            print(actionCounter, ": --------------------------")
            print("action = ", action)
            print("observation = ", observation)
            print("reward = ", reward)
            print("totalReward = ", totalReward)
            print("terminated = ", terminated)
            print("truncated = ", truncated)
            print()

            if terminated or truncated:
                break
            else:
                time.sleep(0.03)
                action = int(mlp.predict(observation.reshape(1, -1)) > 0)

        self.env.close()

In [ ]:
def eaSimpleWithElitism(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    """This algorithm is similar to DEAP eaSimple() algorithm, with the modification that
    halloffame is used to implement an elitism mechanism. The individuals contained in the
    halloffame are directly injected into the next generation and are not subject to the
    genetic operators of selection, crossover and mutation.
    """
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is None:
        raise ValueError("halloffame parameter must not be empty!")

    halloffame.update(population)
    hof_size = len(halloffame.items) if halloffame.items else 0

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):

        # Select the next generation individuals
        offspring = toolbox.select(population, len(population) - hof_size)

        # Vary the pool of individuals
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # add the best back to population:
        offspring.extend(halloffame.items)

        # Update the hall of fame with the generated individuals
        halloffame.update(offspring)

        # Replace the current population by the offspring
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook


In [ ]:
POPULATION_SIZE = 20
P_CROSSOVER = 0.9  # probability for crossover
P_MUTATION = 0.5   # probability for mutating an individual
MAX_GENERATIONS = 10
HALL_OF_FAME_SIZE = 3
CROWDING_FACTOR = 10.0  # crowding factor for crossover and mutation

# set the random seed:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

INPUTS = 4
HIDDEN_LAYER = 4
OUTPUTS = 1


cartPole = CartPole(RANDOM_SEED)


In [ ]:
NUM_OF_PARAMS = len(cartPole)
# boundaries for layer size parameters:

# weight and bias values are bound between -1 and 1:
BOUNDS_LOW, BOUNDS_HIGH = -1.0, 1.0  # boundaries for all dimensions

toolbox = base.Toolbox()

# define a single objective, maximizing fitness strategy:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# create the Individual class based on list:
creator.create("Individual", list, fitness=creator.FitnessMax)

# helper function for creating random real numbers uniformly distributed within a given range [low, up]
# it assumes that the range is the same for every dimension
def randomFloat(low, up):
    return [random.uniform(l, u) for l, u in zip([low] * NUM_OF_PARAMS, [up] * NUM_OF_PARAMS)]

# create an operator that randomly returns a float in the desired range:
toolbox.register("attrFloat", randomFloat, BOUNDS_LOW, BOUNDS_HIGH)

# create an operator that fills up an Individual instance:
toolbox.register("individualCreator",
                 tools.initIterate,
                 creator.Individual,
                 toolbox.attrFloat)

# create an operator that generates a list of individuals:
toolbox.register("populationCreator",
                 tools.initRepeat,
                 list,
                 toolbox.individualCreator)


# fitness calculation using the CrtPole class:
def score(individual):
    return cartPole.getScore(individual),


toolbox.register("evaluate", score)

population_test = toolbox.populationCreator(n=10)
print(population_test)



[[0.2788535969157675, -0.9499784895546661, -0.4499413632617615, -0.5535785237023545, 0.4729424283280248, 0.3533989748458226, 0.7843591354096908, -0.8261223347411677, -0.15615636062945915, -0.9404055611238593, -0.5627240503927933, 0.010710576206724776, -0.9469280606322728, -0.602324698626703, 0.2997688755590464, 0.08988296120643335, -0.5591187559186066, 0.17853136775181744, 0.6188609133556533, -0.987002480643878, 0.6116385036656158, 0.3962787899764537, -0.31949896696401625, -0.6890410003764369, 0.9144261444135624], [-0.32681090977474647, -0.8145083132397042, -0.806567246333072, 0.6949887326949196, 0.20745206273378214, 0.6142565465487604, 0.45946357338763577, 0.07245618290940148, 0.9462315279587412, -0.24293124558329304, 0.104081262546454, 0.6588093285059897, 0.2370395047284921, 0.7234138006215545, 0.15470429051352408, 0.40914367242984695, -0.9083512326886756, -0.5442034486969063, -0.42122407279578566, -0.840416046152745, -0.5344182272779396, -0.7979971411805418, -0.44405279377981577, 0.

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
toolbox.register("select", tools.selTournament, tournsize=2)

toolbox.register("mate",
                 tools.cxSimulatedBinaryBounded,
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR)


toolbox.register("mutate",
                 tools.mutPolynomialBounded,
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR,
                 indpb=1.0/NUM_OF_PARAMS)



In [ ]:
def main():

    # create initial population (generation 0):
    population = toolbox.populationCreator(n=POPULATION_SIZE)

    # prepare the statistics object:
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("max", np.max)
    stats.register("avg", np.mean)

    # define the hall-of-fame object:
    hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

    # perform the Genetic Algorithm flow with hof feature added:
    population, logbook = eaSimpleWithElitism(population,
                                                      toolbox,
                                                      cxpb=P_CROSSOVER,
                                                      mutpb=P_MUTATION,
                                                      ngen=MAX_GENERATIONS,
                                                      stats=stats,
                                                      halloffame=hof,
                                                      verbose=True)

    # print best solution found:
    best = hof.items[0]
    print()
    print("Best Solution = ", best)
    print("Best Score = ", best.fitness.values[0])
    print()

    # save best solution for a replay:
    cartPole.saveParams(best)

    # find average score of 100 episodes using the best solution found:
    print("Running 100 episodes using the best solution...")
    scores = []
    for test in range(100):
        scores.append(CartPole().getScore(best))
    print("scores = ", scores)
    print("Avg. score = ", sum(scores) / len(scores))

In [ ]:
main()

gen	nevals	max	avg  
0  	20    	43 	12.15
1  	17    	43 	12.65
2  	16    	58 	18.7 
3  	14    	237	33.55
4  	15    	237	49.75
5  	14    	500	106.6
6  	17    	237	74.75
7  	15    	500	106.95
8  	16    	500	154.5 
9  	16    	500	163.2 
10 	16    	500	206.1 

Best Solution =  [0.6130654930683185, -0.7134655882081957, -0.7705681528448636, 0.9535119248563795, 0.5208146893095162, -0.05504963480545966, -0.6979945316724185, -0.863748431016116, 0.06417814957311774, -0.9310388305294168, -0.8643873140423275, 0.25154840233470943, -0.3064245073944085, -0.5574234817667543, -0.19644161504343294, -0.557133226567631, -0.7777507488459608, -0.35242256773426484, -0.39261779667873065, -0.6174282061754452, -0.010978643958111212, 0.08453718435504687, -0.32977783093668955, 0.3523748655790124, 0.2530372437846518]
Best Score =  500.0

Running 100 episodes using the best solution...
scores =  [76.0, 123.0, 12.0, 500.0, 60.0, 500.0, 26.0, 172.0, 87.0, 500.0, 252.0, 500.0, 30.0, 94.0, 500.0, 500.0, 500.0, 118.0, 5

In [ ]:
def undx(population, alpha, beta, lower_bound, upper_bound):
  candidates = random.sample(population, 3)
  candidates = np.array(candidates)

  vector12 = candidates[1] - candidates[0]
  origin_12 = (candidates[0] + candidates[1]) / 2
  norm12 = np.linalg.norm(vector12, ord=2)

  sigma1 = alpha * norm12

  vector13 = candidates[2] - candidates[0]
  dot12_13 = np.dot(vector12, vector13)
  vertical13 = vector13 - (dot12_13 / (np.linalg.norm(vector12, ord=2) + 1e-10) ** 2) * vector12

  norm_vertical13 = np.linalg.norm(vertical13, ord=2)
  sigma2 = beta * norm_vertical13 / len(candidates[0])

  noise_horizontal = np.random.normal(0, sigma1, len(candidates[0]))
  noise_vertical = np.random.normal(0, sigma2,len(candidates[0]))


  noised_child = origin_12 + noise_horizontal + noise_vertical

  noise_child = np.clip(noised_child, lower_bound, upper_bound)


  return noised_child


def undx_population(population, alpha, beta, lower_bound, upper_bound, cxpb):
  offspring = []
  population_clone = population.copy()

  while len(offspring) < len(population):
    prob = random.random()

    if prob <= cxpb:
      child = undx(population, alpha, beta, lower_bound, upper_bound)
      child = creator.Individual(child)
      offspring.append(child)
    else:
      candidate = random.choice(population_clone)
      child = toolbox.clone(candidate)
      offspring.append(child)
      population_clone.remove(candidate)

  return offspring



toolbox.register("select", tools.selTournament, tournsize=2)


toolbox.register("mutate",
                 tools.mutPolynomialBounded,
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR,
                 indpb=1.0/NUM_OF_PARAMS)

toolbox.register("mate",
                 undx_population,
                 alpha=0.5,
                 beta=0.35,
                 lower_bound=BOUNDS_LOW,
                 upper_bound=BOUNDS_HIGH,
                 cxpb=P_CROSSOVER
                 )



In [ ]:
population_test = toolbox.populationCreator(n=10)
print(population_test)
print(len(population_test))

offspring = toolbox.mate(population_test)
print(offspring)
print(len(offspring))

[[0.13619964101817295, -0.28586329003099586, 0.5797078144136181, -0.7686196699550452, -0.24033829137418694, -0.5078013331757878, 0.9130375933474699, 0.22688864234965034, 0.3200492729846227, 0.676638327659042, 0.8220271056888131, -0.056926422027363754, -0.6365784636596481, -0.9436539284101082, -0.6831840940609237, 0.1890026840839416, 0.9935369307653503, 0.3003138360114441, -0.18028153472116615, -0.31376914807986345, -0.5012519911678841, -0.6458216529899834, -0.8355863932116183, 0.6144758649898681, -0.48715050416159644], [0.634000431072643, 0.36591530460430177, -0.8469132030760957, 0.7866026923782095, 0.24690368770325133, 0.39313571843247375, 0.917799263429488, -0.3533403778879942, 0.783287334512736, -0.33047578431050173, 0.7443847778161656, 0.2696269738532022, -0.4007738171578239, 0.6564709732771066, -0.914292604099022, -0.25339004452028036, -0.7511205518622188, -0.6221812416720567, 0.12043302576039627, -0.9975551644657714, -0.019170090852242527, -0.5720135717351353, -0.4718824752761437

In [ ]:
def eaSimpleWithElitism_undx(population, toolbox, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    """This algorithm is similar to DEAP eaSimple() algorithm, with the modification that
    halloffame is used to implement an elitism mechanism. The individuals contained in the
    halloffame are directly injected into the next generation and are not subject to the
    genetic operators of selection, crossover and mutation.
    """
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is None:
        raise ValueError("halloffame parameter must not be empty!")

    halloffame.update(population)
    hof_size = len(halloffame.items) if halloffame.items else 0

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):

        # Select the next generation individuals
        offspring = toolbox.select(population, len(population) - hof_size)

        # # Vary the pool of individuals
        # offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)

        #crossover
        offspring = toolbox.mate(offspring)

        #mutation
        for child in offspring:
          if random.random() < mutpb:
            toolbox.mutate(child)
            del child.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # add the best back to population:
        offspring.extend(halloffame.items)

        # Update the hall of fame with the generated individuals
        halloffame.update(offspring)

        # Replace the current population by the offspring
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook

In [ ]:
def main_undx():

    # create initial population (generation 0):
    population = toolbox.populationCreator(n=POPULATION_SIZE)

    # prepare the statistics object:
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("max", np.max)
    stats.register("avg", np.mean)

    # define the hall-of-fame object:
    hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

    # perform the Genetic Algorithm flow with hof feature added:
    population, logbook = eaSimpleWithElitism_undx(population,
                                                      toolbox,
                                                      mutpb=P_MUTATION,
                                                      ngen=MAX_GENERATIONS,
                                                      stats=stats,
                                                      halloffame=hof,
                                                      verbose=True)

    # print best solution found:
    best = hof.items[0]
    print()
    print("Best Solution = ", best)
    print("Best Score = ", best.fitness.values[0])
    print()

    # save best solution for a replay:
    cartPole.saveParams(best)

    # find average score of 100 episodes using the best solution found:
    print("Running 100 episodes using the best solution...")
    scores = []
    for test in range(100):
        scores.append(CartPole().getScore(best))
    print("scores = ", scores)
    print("Avg. score = ", sum(scores) / len(scores))

In [ ]:
main_undx()

gen	nevals	max	avg 
0  	20    	43 	12.3
1  	14    	266	28.2


/usr/local/lib/python3.11/dist-packages/deap/tools/mutation.py:86: RuntimeWarning: invalid value encountered in double_scalars
  delta_q = val ** mut_pow - 1.0
/usr/local/lib/python3.11/dist-packages/deap/tools/mutation.py:86: RuntimeWarning: invalid value encountered in double_scalars
  delta_q = val ** mut_pow - 1.0


2  	15    	266	47.5
3  	14    	266	33.55
4  	15    	266	35.4 


/usr/local/lib/python3.11/dist-packages/deap/tools/mutation.py:86: RuntimeWarning: invalid value encountered in double_scalars
  delta_q = val ** mut_pow - 1.0
/usr/local/lib/python3.11/dist-packages/deap/tools/mutation.py:90: RuntimeWarning: invalid value encountered in double_scalars
  delta_q = 1.0 - val ** mut_pow


5  	16    	266	34.8 
6  	15    	266	35.55
7  	17    	266	35.05


/usr/local/lib/python3.11/dist-packages/deap/tools/mutation.py:86: RuntimeWarning: invalid value encountered in double_scalars
  delta_q = val ** mut_pow - 1.0
/usr/local/lib/python3.11/dist-packages/deap/tools/mutation.py:90: RuntimeWarning: invalid value encountered in double_scalars
  delta_q = 1.0 - val ** mut_pow
/usr/local/lib/python3.11/dist-packages/deap/tools/mutation.py:86: RuntimeWarning: invalid value encountered in double_scalars
  delta_q = val ** mut_pow - 1.0


8  	16    	266	31.2 
9  	15    	266	44.2 


/usr/local/lib/python3.11/dist-packages/deap/tools/mutation.py:86: RuntimeWarning: invalid value encountered in double_scalars
  delta_q = val ** mut_pow - 1.0


10 	17    	266	49.25

Best Solution =  [-0.6761697654550975, -0.9241002389559669, -0.5107769617737523, 0.1691963370624177, 0.9025677041923473, 0.6552817713590952, -0.02748278249919712, -0.14713559719277217, 0.49731143078841, 0.25841217338542644, 0.45803355884780245, -0.0457841571488374, -0.666068746910607, 0.4797296577677943, -0.35481206396258075, 0.2890527457880257, -0.49770121568210984, 0.7690072815364849, 0.5829020864866526, 0.3295670760092718, -0.016188061809456727, -0.007192242686795526, 0.13749377953309938, 0.8290962456207901, -0.3459943225527927]
Best Score =  266.0

Running 100 episodes using the best solution...
scores =  [179.0, 95.0, 108.0, 64.0, 85.0, 113.0, 99.0, 79.0, 95.0, 78.0, 90.0, 78.0, 74.0, 93.0, 145.0, 85.0, 107.0, 91.0, 100.0, 176.0, 83.0, 128.0, 72.0, 53.0, 107.0, 78.0, 74.0, 78.0, 82.0, 126.0, 153.0, 96.0, 164.0, 71.0, 62.0, 112.0, 63.0, 62.0, 142.0, 89.0, 93.0, 64.0, 96.0, 92.0, 99.0, 76.0, 99.0, 82.0, 192.0, 93.0, 90.0, 155.0, 70.0, 107.0, 91.0, 97.0, 104.0, 